In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import seaborn as sns

data = pd.read_csv('/kaggle/input/clothing-dataset-full/images.csv')
data.loc[data['label']=='Not sure','label'] = 'Not_sure'
#data['image'] = data['image'] + '.jpg'

labels=data['label']
features=data.drop('label',1)

labels_values=labels.value_counts()
print(labels_values)

# data['label_cat'] = data['label'] + ' ' + data['kids'].astype(str)
# proc_data=data[['image', 'label_cat']]
# print(type(proc_data))

p= sns.countplot(data=data['image'], x=data['label'])

In [ ]:
labels_y=data.label
labels_num1=labels_y[:100]
labels_num1

In [ ]:
labels_unique=labels_num1.unique()
print(labels_unique.shape)
# data.loc[(data.label) & (data.image == '4285fab0-751a-4b74-8e9b-43af05deee22')]

In [ ]:
# foods = list(os.walk('../input/food41/images/'))[0][1]
nr_foods = 17
labels_values=list(labels_values)

idx_to_name = {i:x for (i,x) in enumerate(labels_unique[:nr_foods])}
name_to_idx = {x:i for (i,x) in enumerate(labels_unique[:nr_foods])}

idx_to_name

In [ ]:
import cv2
from PIL import Image
from tqdm import tqdm 

data_l = []
img_size = (128, 128)

for img in tqdm(data['image'][:100]):
    imgs = [cv2.resize(cv2.imread(f'/kaggle/input/clothing-dataset-full/images_original/{img}.jpg'), img_size, interpolation=cv2.INTER_AREA)]
    for i in imgs:
        data_l.append(i)



In [ ]:
labels_y=[]

import numpy as np
for i in labels_num1:
    labels_y.append(name_to_idx[i])
len(labels_y)

In [ ]:
labels_y_s=pd.Series(labels_y)
len(labels_y_s)

In [ ]:
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

from sklearn.preprocessing import OneHotEncoder


enc = OneHotEncoder()
enc.fit(labels_y_s.values.reshape(-1, 1))

y_transformed = enc.transform(labels_y_s.values.reshape(-1, 1))

import scipy
y_one_hot=scipy.sparse.csr_matrix.toarray(y_transformed)

y_one_hot.shape

In [ ]:
data_l_1 = np.array(data_l)
data_l_2 = data_l_1 / 255
data_l_3 = data_l_2.astype('float32')

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_l_3, y_one_hot,
                                                    test_size=0.2,
                                                    random_state=1,
                                                    shuffle=True)

In [ ]:
def f1(y_true, y_pred): 
    
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    
    return f1_val

In [ ]:
def results_plot(history, title):
    
    fig, ax = plt.subplots(2,1)
    ax[0].plot(history.history['loss'], color='b', label="train loss")
    ax[0].plot(history.history['val_loss'], color='r', label="val loss",axes=ax[0])
    legend = ax[0].legend()

    ax[1].plot(history.history['accuracy'], color='b', label="train acc")
    ax[1].plot(history.history['val_accuracy'], color='r',label="val acc")
    
    legend = ax[1].legend()
    ax[0].set_title(title)
    
    plt.plot()

In [ ]:
def visualize_loss(history, title, clip_to=100):
    
    plt.plot(np.array(history.history['loss']).clip(0,clip_to))
    plt.plot(np.array(history.history['val_loss']).clip(0,clip_to))
    
    plt.title(title)
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper right')
    
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import Sequential, Input, initializers, optimizers, callbacks
from tensorflow.keras.layers import BatchNormalization, Dense, Dropout, Flatten, ReLU, PReLU
from tensorflow.keras.metrics import AUC
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping
seed=47

# **Baseline Model**

In [ ]:
model_1=tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32,kernel_size=(3,3),strides=(1,1),activation='relu',input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(64,kernel_size=(3,3),strides=(1,1),activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(64,kernel_size=(3,3),strides=(1,1),activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(17,activation='softmax')
])
model_1.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics = [tf.keras.metrics.CategoricalAccuracy(name='accuracy'),f1])

history = model_1.fit(X_train,y_train, validation_data=(X_test,y_test),epochs=5,
                    batch_size=32)

In [ ]:
results_plot(history, 'Baseline Model')

In [ ]:
visualize_loss(history,'Baseline model')

# **Model with changed filters' dimention**

In [ ]:
# More dimentions
model_2=tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32,kernel_size=(3,3),strides=(1,1),activation='relu',input_shape=(128,128,3)),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(64,kernel_size=(3,3),strides=(1,1),activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    # change filter size
    tf.keras.layers.Conv2D(128,kernel_size=(3,3),strides=(1,1),activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(17,activation='softmax')
])
model_2.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics = [tf.keras.metrics.CategoricalAccuracy(name='accuracy'),f1])

history_2 = model_2.fit(X_train,y_train, validation_data=(X_test,y_test),epochs=5,
                    batch_size=32)

In [ ]:
results_plot(history_2, 'Model with more dimentions')

In [ ]:
visualize_loss(history_2,'Model with more dimentions')

In [ ]:
# Less dimentions
model_3=tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32,kernel_size=(3,3),strides=(1,1),activation='relu',input_shape=(128,128,3)),
    tf.keras.layers.MaxPool2D(2,2),
    # change filter size
    tf.keras.layers.Conv2D(32,kernel_size=(3,3),strides=(1,1),activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(64,kernel_size=(3,3),strides=(1,1),activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(17,activation='softmax')
])
model_3.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics = [tf.keras.metrics.CategoricalAccuracy(name='accuracy'),f1])

history_3 = model_3.fit(X_train,y_train, validation_data=(X_test,y_test),epochs=5,
                    batch_size=32)

In [ ]:
results_plot(history_3, 'Model with less dimentions')

In [ ]:
visualize_loss(history_3,'Model with less dimentions')

# **Model with changed kernel_size**

In [ ]:
# Higher
model_4=tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32,kernel_size=(4,4),strides=(1,1),activation='relu',input_shape=(128,128,3)),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(64,kernel_size=(4,4),strides=(1,1),activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(64,kernel_size=(4,4),strides=(1,1),activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(17,activation='softmax')
])
model_4.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics = [tf.keras.metrics.CategoricalAccuracy(name='accuracy'),f1])

history_4 = model_4.fit(X_train,y_train, validation_data=(X_test,y_test),epochs=5,
                    batch_size=32)

In [ ]:
results_plot(history_4, 'Model with higher kernel_size')

In [ ]:
visualize_loss(history_4,'Model with higher kernel_size')

In [ ]:
# Lower
model_5=tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32,kernel_size=(2,2),strides=(1,1),activation='relu',input_shape=(128,128,3)),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(64,kernel_size=(2,2),strides=(1,1),activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(64,kernel_size=(2,2),strides=(1,1),activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(17,activation='softmax')
])
model_5.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics = [tf.keras.metrics.CategoricalAccuracy(name='accuracy'),f1])

history_5 = model_5.fit(X_train,y_train, validation_data=(X_test,y_test),epochs=5,
                    batch_size=32)

In [ ]:
results_plot(history_5, 'Model with lower kernel_size')

In [ ]:
visualize_loss(history_5,'Model with lower kernel_size')

# **Model with changed strides**

In [ ]:
model_6=tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32,kernel_size=(3,3),strides=(2,2),activation='relu',input_shape=(128,128,3)),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(64,kernel_size=(3,3),strides=(2,2),activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(64,kernel_size=(3,3),strides=(2,2),activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(17,activation='softmax')
])
model_6.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics = [tf.keras.metrics.CategoricalAccuracy(name='accuracy'),f1])

history_6 = model_6.fit(X_train,y_train, validation_data=(X_test,y_test),epochs=5,
                    batch_size=32)

In [ ]:
 results_plot(history_6, 'Model with lower kernel_size')

In [ ]:
visualize_loss(history_6,'Model with lower kernel_size')

# **Model with changed padding**

In [ ]:
# Higher
model_7=tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32,kernel_size=(4,4),strides=(1,1),activation='relu',padding='valid', input_shape=(128,128,3)),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(64,kernel_size=(4,4),strides=(1,1),padding='valid',activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(64,kernel_size=(4,4),strides=(1,1),padding='valid',activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(17,activation='softmax')
])
model_7.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics = [tf.keras.metrics.CategoricalAccuracy(name='accuracy'),f1])

history_7 = model_7.fit(X_train,y_train, validation_data=(X_test,y_test),epochs=5,
                    batch_size=32)

In [ ]:
results_plot(history_7, 'Model with lower kernel_size')

In [ ]:
visualize_loss(history_7,'Model with lower kernel_size')

# **Model with batch normalization**

In [ ]:
# Higher
model_8=tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32,kernel_size=(4,4),strides=(1,1),activation='relu',padding='valid', input_shape=(128,128,3)),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64,kernel_size=(4,4),strides=(1,1),padding='valid',activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64,kernel_size=(4,4),strides=(1,1),padding='valid',activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(17,activation='softmax')
])
model_8.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics = [tf.keras.metrics.CategoricalAccuracy(name='accuracy'),f1])

history_8 = model_8.fit(X_train,y_train, validation_data=(X_test,y_test),epochs=5,
                    batch_size=32)

In [ ]:
results_plot(history_8, 'Model with lower kernel_size')

In [ ]:
visualize_loss(history_8,'Model with lower kernel_size')

# **Model with average pooling**

In [ ]:
#Higher
model_9=tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32,kernel_size=(4,4),strides=(1,1),activation='relu',padding='valid', input_shape=(128,128,3)),
    tf.keras.layers.AveragePooling2D(2,2),
    tf.keras.layers.Conv2D(64,kernel_size=(4,4),strides=(1,1),padding='valid',activation='relu'),
    tf.keras.layers.AveragePooling2D(2,2),
    tf.keras.layers.Conv2D(64,kernel_size=(4,4),strides=(1,1),padding='valid',activation='relu'),
    tf.keras.layers.AveragePooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(17,activation='softmax')
])
model_9.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics = [tf.keras.metrics.CategoricalAccuracy(name='accuracy'),f1])

history_9 = model_9.fit(X_train,y_train, validation_data=(X_test,y_test),epochs=5,
                    batch_size=32)

In [ ]:
results_plot(history_9, 'Model with lower kernel_size')

In [ ]:
visualize_loss(history_9,'Model with lower kernel_size')

# **Model with dropout 0.2**

In [ ]:
# Higher
model_10=tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32,kernel_size=(4,4),strides=(1,1),activation='relu',padding='valid', input_shape=(128,128,3)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(64,kernel_size=(4,4),strides=(1,1),padding='valid',activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(64,kernel_size=(4,4),strides=(1,1),padding='valid',activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(17,activation='softmax')
])
model_10.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics = [tf.keras.metrics.CategoricalAccuracy(name='accuracy'),f1])

history_10 = model_10.fit(X_train,y_train, validation_data=(X_test,y_test),epochs=5,
                    batch_size=32)

In [ ]:
results_plot(history_10, 'Model with lower kernel_size')

In [ ]:
visualize_loss(history_10,'Model with lower kernel_size')

# **Model with dropout 0.4**

In [ ]:
# Higher
model_11=tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32,kernel_size=(4,4),strides=(1,1),activation='relu',padding='valid', input_shape=(128,128,3)),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(64,kernel_size=(4,4),strides=(1,1),padding='valid',activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(64,kernel_size=(4,4),strides=(1,1),padding='valid',activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(17,activation='softmax')
])
model_11.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics = [tf.keras.metrics.CategoricalAccuracy(name='accuracy'),f1])

history_11 = model_11.fit(X_train,y_train, validation_data=(X_test,y_test),epochs=5,
                    batch_size=32)

In [ ]:
results_plot(history_11, 'Model with lower kernel_size')

In [ ]:
visualize_loss(history_11,'Model with lower kernel_size')

# **Best Model**

In [ ]:
# More dimentions
model_2=tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32,kernel_size=(3,3),strides=(1,1),activation='relu',input_shape=(128,128,3)),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(64,kernel_size=(3,3),strides=(1,1),activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    # change filter size
    tf.keras.layers.Conv2D(128,kernel_size=(3,3),strides=(1,1),activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(17,activation='softmax')
])
model_2.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics = [tf.keras.metrics.CategoricalAccuracy(name='accuracy'),f1])

history_2 = model_2.fit(X_train,y_train, validation_data=(X_test,y_test),epochs=5,
                    batch_size=32)